In [1]:
using System.Net;
using System;
using System.Net.Http;

async Task<string> GetInputAsync(int day)
{
    var url = $"https://adventofcode.com/2021/day/{day}/input";
    var secret = Environment.GetEnvironmentVariable("AOC_SECRET");
    if (string.IsNullOrEmpty(secret))
    {
        throw new InvalidOperationException("Need to set AOC_SECRET");
    }
    
    var cookieContainer = new CookieContainer();
    using var handler = new HttpClientHandler() { CookieContainer = cookieContainer };
    using var client = new HttpClient(handler);

    var cookie = new Cookie("session", secret);
    cookie.Domain = "adventofcode.com";
    cookieContainer.Add(cookie);
    var result = await client.GetAsync(url);
    return await result.Content.ReadAsStringAsync();
}

In [13]:
// Day 1

var input = await GetInputAsync(1);
var lines = input.Split("\n", StringSplitOptions.RemoveEmptyEntries);

// Part 1
var measurements = lines.Select(l => int.Parse(l)).ToArray();
var increases = 0;
for (var i = 1; i < measurements.Length; i++)
{
    if (measurements[i] > measurements[i-1])
    {
        increases++;
    }
}

Console.WriteLine($"Part 1: Incresed {increases} times");

// Part 2
var windowSize = 3;
increases = 0;
for (var i = 1; i < measurements.Length - (windowSize - 1); i++)
{
    var thisWindow = measurements[i..(i+3)];
    var previousWindow = measurements[(i-1)..(i+2)];
    if (thisWindow.Sum() > previousWindow.Sum())
    {
        increases++;
    }
}

Console.WriteLine($"Part 2: Incresed {increases} times");

Part 1: Incresed 1154 times
Part 2: Incresed 1127 times
